In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

In [35]:
class DeformableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, deformable_groups=1):
        super(DeformableConv2D, self).__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding)
        self.offset_conv = nn.Conv2d(in_channels, 2 * kernel_size * kernel_size, kernel_size, stride=stride, padding=padding)
        self.deformable_groups = deformable_groups

    def forward(self, x):
        offset = self.offset_conv(x)

        N, C, H, W = offset.size()
        offset = offset.view(N, 2 ,self.deformable_groups  , H * W)
        print(offset.size())
        offset = offset.permute(0, 2, 3, 1).contiguous()
        print(offset.size())

        x_offset = F.grid_sample(x, offset)
        return self.conv(x_offset)
# معماری شبکه DCN با لایه Deformable Convolution دستی
class DeformableCNN(nn.Module):
    def __init__(self):
        super(DeformableCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.deformable_conv = DeformableConv2D(32, 64, kernel_size=3, deformable_groups=9)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.deformable_conv(x))
        x = x.view(-1, 64 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [36]:
batch_size = 64
learning_rate = 0.001
epochs = 5

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

deformable_cnn = DeformableCNN()
criterion = nn.CrossEntropyLoss()
optimizer_deformable = optim.Adam(deformable_cnn.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(1, epochs + 1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer_deformable.zero_grad()
        output = deformable_cnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer_deformable.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
deformable_cnn.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        output = deformable_cnn(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))